In [4]:
#Import Libraries and Package Modules

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
#Defining a Function to Download MNIST Dataset

def downloadDatasets():
    
    #Dataset for Training the Model
    trainData = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )
    
    #Dataset for Testing/ Validating the Model
    validationData = datasets.MNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    return trainData, validationData

In [6]:
#Creating Script to Check downloadDataset()

if __name__ == "__main__":

    # download MNIST dataset
    trainData, _ = downloadDatasets()
    print("Downloaded MNISt Dataset")

Downloaded MNISt Dataset


In [7]:
#Create a DataLoader for the Train set
BATCH_SIZE = 128

def createDataLoader(trainData, batchSize):
    trainDataloader = DataLoader(trainData, batch_size=batchSize)
    return trainDataloader

trainDataLoader = createDataLoader(trainData, BATCH_SIZE)
print("DataLoader for Train Dataset is Created.")

DataLoader for Train Dataset is Created.


In [8]:
#Built a Model - Feed Forward Network

class FeedForwardNet(nn.Module):
    #Creating Constructor
    def __init__(self):
        #Invoke Cnstructor of base class
        super().__init__()
        #Create Initial Layer
        self.flatten = nn.Flatten()
        #Create Dense Layers
        #Use Sequential() to pack up Dense layers together
        self.denseLayers = nn.Sequential(
            #Appling Linear Layer
            nn.Linear(28 * 28, 256),
            #Applying Activation Function: Rectified Linear Unit
            nn.ReLU(),
            #Another Linear Function
            nn.Linear(256, 10)
        )
        #Perform Basic Transformation using softmax()
        self.softmax = nn.Softmax(dim=1)
    
    #Defining Forward Function
    def forward(self, input_data):
        #Get Flattened Data
        x = self.flatten(input_data)
        #Get Logits
        logits = self.denseLayers(x)
        #Get Predictions
        predictions = self.softmax(logits)
        return predictions


In [9]:
#Updating Script to Check Feed Forward Network Model Working

if __name__ == "__main__":

    # download MNIST dataset
    trainData, _ = downloadDatasets()
    #print("Downloaded MNISt Dataset")
    trainDataLoader = createDataLoader(trainData, BATCH_SIZE)

    #Check which device I'm using
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")
    # construct model and assign it to device
    feedForwardNet = FeedForwardNet().to(device)
    print(feedForwardNet)

Using cpu
FeedForwardNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (denseLayers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)


In [10]:
def trainOneEpoch(model, dataLoader, lossFn, optimiser, device):
    for input, target in dataLoader:
        input, target = input.to(device), target.to(device)

        # calculate loss
        prediction = model(input)
        loss = lossFn(prediction, target)

        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    print(f"Loss: {loss.item()}")


def train(model, dataLoader, lossFn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        trainOneEpoch(model, dataLoader, lossFn, optimiser, device)
        print("---------------------------")
    print("Finished training")

In [14]:
   
#Defining Contants
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001    

#Updating Script to Check Feed Forward Network Model Working
if __name__ == "__main__":

    # download data and create data loader
    trainData, _ = downloadDatasets()
    trainDataLoader = createDataLoader(trainData, BATCH_SIZE)

    #Construct Model and Assign it to device
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")
    feedForwardNet = FeedForwardNet().to(device)
    print(feedForwardNet)

    #Initialise loss funtion + optimiser
    lossFn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(feedForwardNet.parameters(),
                                 lr=LEARNING_RATE)

    #Train Model
    train(feedForwardNet, trainDataLoader, lossFn, optimiser, device, EPOCHS)

    #Save Model
    torch.save(feedForwardNet.state_dict(), "feedforwardnet.pth")
    print("Trained feed forward net saved at feedforwardnet.pth")

Using cpu
FeedForwardNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (denseLayers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)
Epoch 1
Loss: 1.5120779275894165
---------------------------
Epoch 2
Loss: 1.4969273805618286
---------------------------
Epoch 3
Loss: 1.4974244832992554
---------------------------
Epoch 4
Loss: 1.4847084283828735
---------------------------
Epoch 5
Loss: 1.4773716926574707
---------------------------
Epoch 6
Loss: 1.4746065139770508
---------------------------
Epoch 7
Loss: 1.4735969305038452
---------------------------
Epoch 8
Loss: 1.4733158349990845
---------------------------
Epoch 9
Loss: 1.472577691078186
---------------------------
Epoch 10
Loss: 1.472303867340088
---------------------------
Finished training
Trained feed forward net saved at feedforwardnet.pth
